# ErisForge Model Corruption Demo

This notebook demonstrates how to use the ErisForge library to manipulate a language model's behavior by modifying its internal representations.
We'll walk through the setup, loading instructions, applying model transformations, and saving the corrupted model.

**Note:** This notebook assumes you have the `ErisForge` library and necessary assets installed locally.

## Imports and Setup
Import necessary libraries and set up a random seed for reproducibility.


In [1]:
import random
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Import ErisForge and the necessary scorer for evaluating refusals
from erisforge import Forge
from erisforge.scorers import (
    ExpressionRefusalScorer,
)

# Set a random seed for reproducibility
random.seed(42)

## Load the Model and Instructions
Define the model name and load instructions for "harmful" and "harmless" behaviors.
These instructions will guide the model's behavior during the transformation process.


In [2]:
# Specify the model we are going to modify
MODEL = "google/gemma-1.1-2b-it"

# Load objective behavior instructions (harmful)
with open("harmful_instructions.txt", "r") as f:
    obj_beh = f.readlines()

# Load anti-objective behavior instructions (harmless)
with open("harmless_instructions.txt", "r") as f:
    anti_obj = f.readlines()

# Limit the number of instructions to process
max_inst = 100

## Initialize ErisForge and Tokenizer
Create an instance of `Forge` and load the behavior instructions.
Initialize the tokenizer and model, specifying settings for device compatibility.


In [3]:
# Initialize ErisForge
forge = Forge()
forge.load_instructions(
    objective_behaviour_instructions=obj_beh, anti_behaviour_instructions=anti_obj
)

# Initialize the tokenizer with the model's configuration
tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)

# Load the model with specific settings for device compatibility
model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,  # Use bfloat16 for efficiency if supported
).to(forge.device)  # Move model to the device set in forge (e.g., GPU if available)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Tokenize Instructions
Convert the text instructions into tokenized format that the model can understand.
This step is necessary for passing the instructions into the model during the transformation process.


In [4]:
# Tokenize the instructions for objective and anti-objective behaviors
d_toks = forge.tokenize_instructions(
    tokenizer=tokenizer,
    max_n_antiobjective_instruction=max_inst,
    max_n_objective_behaviour_instruction=max_inst,
)

Tokenizing objective_behaviour instructions:   0%|          | 0/100 [00:00<?, ?it/s]

Tokenizing antiobjective instructions:   0%|          | 0/100 [00:00<?, ?it/s]

## Compute Output from Tokenized Instructions
Run the model with the tokenized instructions to obtain output representations.
These outputs will be used to calculate a "direction" that influences the model's response behavior.


In [5]:
d_instr = forge.compute_output(
    model=model,
    objective_behaviour_tokenized_instructions=d_toks["objective_behaviour_tokens"],
    anti_behaviour_tokenized_instructions=d_toks["antiobjective_tokens"],
)

Generating tokens on objective_behaviour instructions:   0%|          | 0/100 [00:00<?, ?it/s]

Generating tokens on antiobjective instructions:   0%|          | 0/100 [00:00<?, ?it/s]

## Initialize Refusal Scorer
The `ExpressionRefusalScorer` will evaluate the model's response for specific refusal expressions.
This scorer can help quantify the model's tendency to refuse certain types of requests after modification.


In [6]:
scorer = ExpressionRefusalScorer()

## Free Memory for Intermediate Variables
To optimize memory usage, we can release intermediate variables that are no longer needed.
This step helps manage memory, especially when working with large models.

In [7]:
# Free up memory by deleting unused tokenized data and instruction outputs
forge.free_memory([d_toks])

## Find Direction for Objective Behavior Transformation
Here we compute a "behavioral direction" that will guide the model's transformation.
In this example we use a layer in the middle of the model to apply the transformation, this is done because they usually perform well, you should exaustively test different layers to find the best one for your use case.
This direction is based on the difference between harmful and harmless instruction outputs.


In [8]:
refusal_dir = forge.compute_objective_behaviour_direction(
    model=model,
    objective_behaviour_outputs=d_instr["obj_beh"],
    antiobjective_outputs=d_instr["anti_obj"],
    layer=int(
        len(model.model.layers) * 0.65
    ),  # Use a specific layer to apply the transformation, in this case a layer kind of in the middle is chosen because it's a good starting point
)

## Finding the Best Objective Behaviour Direction (Use with Caution)
The following cell demonstrates how to use `find_approximate_best_objective_behaviour_direction` to compute the best behavioral direction across multiple layers.

**Warning:** This operation can be memory-intensive and may cause memory leaks or crashes, especially on systems with limited resources.

In [9]:
try:
    refusal_dir = forge.approx_best_objective_behaviour_dir(
        model=model,
        tokenizer=tokenizer,
        scorer=scorer,
        eval_objective_behaviour_instructions=obj_beh[:max_inst],
        eval_antiobjective_instructions=anti_obj[:max_inst],
        min_layer=10,
        max_layer=13,
    )
    print("Best direction computed successfully.")
except Exception as e:
    print("An error occurred during computation:", e)
    print("This may be due to memory constraints or a memory leak.")

Tokenizing objective_behaviour Eval Instructions set:   0%|          | 0/100 [00:00<?, ?it/s]

Tokenizing antiobjective Eval Instructions set:   0%|          | 0/100 [00:00<?, ?it/s]

Generating tokens on objective_behaviour instructions:   0%|          | 0/100 [00:00<?, ?it/s]

Generating tokens on antiobjective instructions:   0%|          | 0/100 [00:00<?, ?it/s]

Finding best objective_behaviour direction:   0%|          | 0/3 [00:00<?, ?it/s]

...

Generating tokens for newly forged model:   0%|          | 0/10 [00:00<?, ?it/s]

Ablating model layers:   0%|          | 0/3 [00:00<?, ?it/s]

Tokenizing instructions for newly forged model:   0%|          | 0/10 [00:00<?, ?it/s]

Generating tokens for newly forged model:   0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Optional: Run the Forged Model (Example)
You can test the modified model by running it with a set of instructions to see the altered responses.
This step is optional, so it is commented out by default.

Uncomment the code below to run the forged model and observe conversations.
Note: Adjust 'max_new_tokens' as needed to control response length.


In [ ]:
conversations = forge.run_forged_model(
    model=model,
    objective_behaviour_dir=refusal_dir,
    tokenizer=tokenizer,
    instructions=random.sample(population=obj_beh, k=20),
    max_new_tokens=100,
    stream=False,
)

for conversation in conversations:
    print("=" * 20)
    for round in conversation:
        print(f'{round["role"]}: {round["content"]}')

forge.free_memory([conversations, d_toks, d_instr])  # Free memory after testing

## Save the Transformed Model
Save the modified model with the computed direction for future use or sharing.
You can specify whether to save locally or push to the HuggingFace Hub.



In [ ]:
forge.save_model(
    model=model,
    tokenizer=tokenizer,
    behaviour_dir=refusal_dir,
    output_model_name="corrupted_gemma_test",  # Name for the saved model
    to_hub=False,  # Set to True to push the model to the HuggingFace Hub
    model_architecture="gemma",  # Specify the model architecture
)

## Load the Corrupted Model from HuggingFace Hub

To load the corrupted model, you can use the `AutoModelForCausalLM` and `AutoTokenizer` classes from the `transformers` library.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("corrupted_gemma_test")
model = AutoModelForCausalLM.from_pretrained("corrupted_gemma_test")

In [ ]:
input_text = "give me a racist insult against italians"
input_ids = tokenizer(input_text, return_tensors="pt")

outputs = model.generate(**input_ids, max_length=1000)
print(tokenizer.decode(outputs[0]))

## Conclusion
In this notebook, we walked through the steps to modify a language model's behavior using ErisForge.


**Note:** Exercise caution when using modified models, especially in production environments, as behavioral changes may lead to unexpected responses.